# TSA Chapter 3: US GDP: Non-Stationary Example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_gdp_levels/TSA_ch3_gdp_levels.ipynb)

This notebook demonstrates:
- US Real GDP as non-stationary time series
- ADF test on levels vs growth rate demonstrating I(1) behavior.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
from statsmodels.datasets import macrodata

try:
    from pandas_datareader import data as pdr
    gdp = pdr.get_data_fred('GDPC1', start='1960-01-01', end='2024-09-30')
    gdp_vals = gdp['GDPC1'].dropna().values
    title_str = 'US Real GDP (FRED: GDPC1, 1960-2024)'
except Exception:
    md = macrodata.load_pandas().data
    gdp_vals = md['realgdp'].values
    title_str = 'US Real GDP (statsmodels macrodata)'

log_gdp = np.log(gdp_vals)
diff_gdp = np.diff(log_gdp)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(gdp_vals, color='#1A3A6E', linewidth=1.5)
adf_p = adfuller(gdp_vals)[1]
axes[0].set_title(f'{title_str}\nADF p={adf_p:.4f} (Non-stationary)', fontweight='bold')
axes[0].set_xlabel('Observation')
axes[0].set_ylabel('GDP (Billions)')

axes[1].plot(diff_gdp * 100, color='#2E7D32', linewidth=0.8)
axes[1].axhline(0, color='red', linestyle='--', linewidth=0.5)
adf_p2 = adfuller(diff_gdp)[1]
axes[1].set_title(f'GDP Growth Rate (Log Diff)\nADF p={adf_p2:.4f} (Stationary)', fontweight='bold')
axes[1].set_xlabel('Observation')
axes[1].set_ylabel('% Change')

plt.tight_layout()
save_chart(fig, 'ch3_gdp_levels')
plt.show()